In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("facebook/xlm-roberta-xl")
model = AutoModelForMaskedLM.from_pretrained("facebook/xlm-roberta-xl")

tokenizer_config.json:   0%|          | 0.00/405 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/13.9G [00:00<?, ?B/s]

In [3]:
model

XLMRobertaXLForMaskedLM(
  (roberta): XLMRobertaXLModel(
    (embeddings): XLMRobertaXLEmbeddings(
      (word_embeddings): Embedding(250880, 2560, padding_idx=1)
      (position_embeddings): Embedding(514, 2560, padding_idx=1)
      (token_type_embeddings): Embedding(1, 2560)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaXLEncoder(
      (layer): ModuleList(
        (0-35): 36 x XLMRobertaXLLayer(
          (attention): XLMRobertaXLAttention(
            (self_attn_layer_norm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
            (self): XLMRobertaXLSdpaSelfAttention(
              (query): Linear(in_features=2560, out_features=2560, bias=True)
              (key): Linear(in_features=2560, out_features=2560, bias=True)
              (value): Linear(in_features=2560, out_features=2560, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaXLSelfOutput(
              (dense): Linear(i

In [4]:
# Set the model to evaluation mode
model.eval()



XLMRobertaXLForMaskedLM(
  (roberta): XLMRobertaXLModel(
    (embeddings): XLMRobertaXLEmbeddings(
      (word_embeddings): Embedding(250880, 2560, padding_idx=1)
      (position_embeddings): Embedding(514, 2560, padding_idx=1)
      (token_type_embeddings): Embedding(1, 2560)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaXLEncoder(
      (layer): ModuleList(
        (0-35): 36 x XLMRobertaXLLayer(
          (attention): XLMRobertaXLAttention(
            (self_attn_layer_norm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
            (self): XLMRobertaXLSdpaSelfAttention(
              (query): Linear(in_features=2560, out_features=2560, bias=True)
              (key): Linear(in_features=2560, out_features=2560, bias=True)
              (value): Linear(in_features=2560, out_features=2560, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaXLSelfOutput(
              (dense): Linear(i

In [8]:
# Define your sentences
sentence_en = "It is hot today."
sentence_ko = "이것은 한국어로 된 예문입니다."

# Function to encode sentences and get embeddings
def get_embedding(sentence):
    # Tokenize the sentence
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)
    
    # Get the model output
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Extract the last hidden state
    # We can use the output of the [CLS] token as the sentence embedding
    embeddings = outputs[0][:, 0, :]  # [CLS] token representation
    return embeddings

# Get embeddings for both sentences
embedding_en = get_embedding(sentence_en)
embedding_ko = get_embedding(sentence_ko)

# Compute cosine similarity
similarity = cosine_similarity(embedding_en.numpy(), embedding_ko.numpy())

print("Cosine Similarity:", similarity[0][0])

Cosine Similarity: 0.99751174


In [10]:
import pandas as pd


df = pd.DataFrame({
    'eng_sent': [sentence_en],
    'kor_sent': [sentence_ko]
})

In [14]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import time

models = [
    
    "xlm-roberta-base",
    "FacebookAI/xlm-roberta-base",
    'paraphrase-multilingual-mpnet-base-v2',
    "paraphrase-multilingual-MiniLM-L12-v2"
]


# List of free models
# models = [
    
#     "xlm-roberta-base"
# ]

# Dictionaries to store results
similarity_results = {model_name: [] for model_name in models}
time_results = {model_name: 0 for model_name in models}

for model_name in models:
    print("-------------------------------------------------")
    print(f"Processing model: {model_name}")
    
    start_time = time.time()  # Start timer
    
    try:
        # Load model
        model = SentenceTransformer(model_name, trust_remote_code=True)
        
        # Encode sentences
        kor_embeddings = model.encode(df['kor_sent'].tolist())
        eng_embeddings = model.encode(df['eng_sent'].tolist())
        
        # Compute similarity scores
        similarity_scores = [cosine_similarity([kor_embeddings[i]], [eng_embeddings[i]])[0, 0] for i in range(len(df))]
        
        # Store similarity scores
        similarity_results[model_name] = similarity_scores
    except Exception as e:
        print(f"Failed to process model {model_name}: {e}")
        similarity_results[model_name] = [None] * len(df)
    finally:
        end_time = time.time()  # End timer
        time_taken = end_time - start_time
        time_results[model_name] = time_taken
        print(f"Time taken: {time_taken:.2f} seconds")
         # Clear the model from memory
        #del model
        #gc.collect()  # Force garbage collection

# Add similarities and time taken as columns to the DataFrame
for model_name, scores in similarity_results.items():
    df[model_name + "_sim"] = scores

for model_name, time_taken in time_results.items():
    df[model_name + "_time"] = time_taken

# Find the model with the highest similarity for each row
df["best_model"] = df[[col for col in df.columns if col.endswith("_sim")]].idxmax(axis=1)

# Clean the column names in `best_model`
df["best_model"] = df["best_model"].str.replace("_sim", "")

# Add the time taken for the best model
df["model_time_taken"] = df["best_model"].map(time_results)

# Add optimal time taken (minimum across all models)
df["optimal_time_taken"] = min(time_results.values())

print(df)


-------------------------------------------------
Processing model: xlm-roberta-base


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Time taken: 35.28 seconds
           eng_sent           kor_sent  xlm-roberta-base_sim  \
0  It is hot today.  이것은 한국어로 된 예문입니다.              0.993354   

   FacebookAI/xlm-roberta-base_sim  paraphrase-multilingual-mpnet-base-v2_sim  \
0                         0.993354                                   0.232731   

   paraphrase-multilingual-MiniLM-L12-v2_sim  xlm-roberta-base_time  \
0                                    0.07898              50.010739   

   FacebookAI/xlm-roberta-base_time  \
0                         43.274361   

   paraphrase-multilingual-mpnet-base-v2_time  \
0                                   20.721071   

   paraphrase-multilingual-MiniLM-L12-v2_time        best_model  \
0                                   35.284058  xlm-roberta-base   

   model_time_taken  optimal_time_taken  
0         50.010739           20.721071  


In [15]:
df

,eng_sent,kor_sent,xlm-roberta-base_sim,FacebookAI/xlm-roberta-base_sim,paraphrase-multilingual-mpnet-base-v2_sim,paraphrase-multilingual-MiniLM-L12-v2_sim,xlm-roberta-base_time,FacebookAI/xlm-roberta-base_time,paraphrase-multilingual-mpnet-base-v2_time,paraphrase-multilingual-MiniLM-L12-v2_time,best_model,model_time_taken,optimal_time_taken
0,It is hot today.,이것은 한국어로 된 예문입니다.,0.993354,0.993354,0.232731,0.07898,50.010739,43.274361,20.721071,35.284058,xlm-roberta-base,50.010739,20.721071


In [13]:
!pip install sentence_transformers

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 4.3 MB/s eta 0:00:00a 0:00:01
